In [1]:
import numpy as np
import pandas as pd # read_csv,get_dumies(원핫인코딩)
from sklearn.model_selection import train_test_split #훈련셋과 시험셋 나누기
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Dropout
from tensorflow.keras.callbacks import EarlyStopping,Callback
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
from pandas import get_dummies

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

- Red Wine 품질 등급 예측

```
1. 데이터 셋 확보 & 전처리
    독립변수, 종속변수 분리 -> 독립 변수 스케일 조정(StandardScaler), 종속변수 원핫 인코딩, 훈련셋과 테스트셋 분리
2. 모델구성(입력11,출력6)
3. 모델 학습 과정 설정(다중분류에 맞는 설정)
4. 학습시키기(callbacks이용)
5. 모델 평가 - 그래프,평가(테스트셋), 교차표
6. 모델 저장 및 모델사용하기
    
```

In [3]:
redwine = pd.read_csv('./data/winequality-red.csv',sep=';')
redwine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [4]:
redwine.quality.unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [39]:
Input= redwine.iloc[:,:-1]
target= redwine.iloc[:,-1]
display(Input)
display(target)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

In [6]:
# scale 조정(StandardScaler)
scaler = StandardScaler()
scaled_input = pd.DataFrame(scaler.fit_transform(Input))
scaled_input.shape

(1599, 11)

In [41]:
# 종속변수 원핫 인코딩(get_dummies)
target = get_dummies(target).to_numpy()

In [42]:
target.shape

(1599, 6)

In [10]:
# 종속변수 원핫 인코딩(get_dummies)
# Y_train = get_dummies(y_train)
# Y_test = get_dummies(y_test)


In [43]:
# scaled_input와 target => 학습셋: 테스트셋 = 0.75:0.25
X_train,X_test,y_train,y_test = train_test_split(scaled_input,
                                                target,
                                                test_size=0.25)


In [36]:
# 모델 생성 & 모델 구성
model=Sequential()
model.add(Dense(units=100, input_dim=11,activation = 'relu'))
model.add(Dense(units=200, activation = 'relu'))
model.add(Dense(units=100, activation = 'relu'))
model.add(Dense(units=1, activation = 'softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 100)               1200      
                                                                 
 dense_29 (Dense)            (None, 200)               20200     
                                                                 
 dense_30 (Dense)            (None, 100)               20100     
                                                                 
 dense_31 (Dense)            (None, 1)                 101       
                                                                 
Total params: 41,601
Trainable params: 41,601
Non-trainable params: 0
_________________________________________________________________


In [37]:
#model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.compile(loss = 'binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])

In [38]:
hist = model.fit(X_train, Y_train,
               epochs=300,
               # batch_size=350,
               # validation_split=0.2 # 검증데이터 비율(훈련데이터셋 중 20%를 검증용 데이터로)
               
               )

Epoch 1/300
38/38 [==============================] - 1s 2ms/step - loss: -75.3820 - binary_accuracy: 0.0000e+00
Epoch 2/300
38/38 [==============================] - 0s 2ms/step - loss: -1443.2261 - binary_accuracy: 0.0000e+00
Epoch 3/300
38/38 [==============================] - 0s 2ms/step - loss: -10649.7666 - binary_accuracy: 0.0000e+00
Epoch 4/300
38/38 [==============================] - 0s 2ms/step - loss: -45419.4102 - binary_accuracy: 0.0000e+00
Epoch 5/300
38/38 [==============================] - 0s 2ms/step - loss: -139628.9844 - binary_accuracy: 0.0000e+00
Epoch 6/300
38/38 [==============================] - 0s 2ms/step - loss: -345063.0000 - binary_accuracy: 0.0000e+00
Epoch 7/300
38/38 [==============================] - 0s 2ms/step - loss: -734440.4375 - binary_accuracy: 0.0000e+00
Epoch 8/300
38/38 [==============================] - 0s 2ms/step - loss: -1396047.0000 - binary_accuracy: 0.0000e+00
Epoch 9/300
38/38 [==============================] - 0s 2ms/step - loss: -24378

Epoch 136/300
38/38 [==============================] - 0s 2ms/step - loss: -79434014720.0000 - binary_accuracy: 0.0000e+00
Epoch 137/300
38/38 [==============================] - 0s 2ms/step - loss: -81311014912.0000 - binary_accuracy: 0.0000e+00
Epoch 138/300
38/38 [==============================] - 0s 2ms/step - loss: -83190210560.0000 - binary_accuracy: 0.0000e+00
Epoch 139/300
38/38 [==============================] - 0s 2ms/step - loss: -85122105344.0000 - binary_accuracy: 0.0000e+00
Epoch 140/300
38/38 [==============================] - 0s 2ms/step - loss: -87071621120.0000 - binary_accuracy: 0.0000e+00
Epoch 141/300
38/38 [==============================] - 0s 2ms/step - loss: -89052856320.0000 - binary_accuracy: 0.0000e+00
Epoch 142/300
38/38 [==============================] - 0s 2ms/step - loss: -91062083584.0000 - binary_accuracy: 0.0000e+00
Epoch 143/300
38/38 [==============================] - 0s 2ms/step - loss: -93115252736.0000 - binary_accuracy: 0.0000e+00
Epoch 144/300
38

38/38 [==============================] - 0s 2ms/step - loss: -673729347584.0000 - binary_accuracy: 0.0000e+00
Epoch 269/300
38/38 [==============================] - 0s 2ms/step - loss: -681753378816.0000 - binary_accuracy: 0.0000e+00
Epoch 270/300
38/38 [==============================] - 0s 2ms/step - loss: -689829773312.0000 - binary_accuracy: 0.0000e+00
Epoch 271/300
38/38 [==============================] - 0s 2ms/step - loss: -697939984384.0000 - binary_accuracy: 0.0000e+00
Epoch 272/300
38/38 [==============================] - 0s 2ms/step - loss: -706166128640.0000 - binary_accuracy: 0.0000e+00
Epoch 273/300
38/38 [==============================] - 0s 2ms/step - loss: -714425171968.0000 - binary_accuracy: 0.0000e+00
Epoch 274/300
38/38 [==============================] - 0s 2ms/step - loss: -722749882368.0000 - binary_accuracy: 0.0000e+00
Epoch 275/300
38/38 [==============================] - 0s 2ms/step - loss: -731159920640.0000 - binary_accuracy: 0.0000e+00
Epoch 276/300
38/38 [=

In [ ]:
# 1번
((X_train,Y_train),(X_test,Y_test))=mnist.load_data()

#훈련셋(6만)= 5만개 +만개(val)
X_val = X_train[50000:]
Y_val = Y_train[50000:]
X_train = X_train[:50000]
Y_train = Y_train[:50000]

# 독립변수 전처리 :(5000,28,28) ->(50000,784)-> 실수형변환 -> 스케일조정(255.00으로나눠)
X_train=X_train.reshape(-1,784).astype('float32')/255.0
X_val = X_val.reshape(-1,784).astype('float32')/255.0
X_test = X_test.reshape(-1,784).astype('float32')/255.0
X_train.shape,X_val.shape,X_test.shape

# 훈련셋(5만)과 검증셋(만)-> 700개,300개만 random으로 choice
train_idxs=np.random.choice(50000,700)
val_idxs=np.random.choice(10000,300)

X_train= X_train[train_idxs]
Y_train= Y_train[train_idxs]
X_val = X_val[val_idxs]
Y_val = Y_val[val_idxs]

# 종속변수 전처리: 원핫 인코딩(to_categorical(넘파이배열)pd.get_dummies(데이터프레임을 반환))
print('원핫인코딩 전 shape:',Y_train.shape,Y_val.shape,Y_test.shape)
#원핫인코딩
Y_train = to_categorical(Y_train,10)
Y_val = to_categorical(Y_val)
Y_test = to_categorical(Y_test)
print('원핫인코딩 전 shape:',Y_train.shape,Y_val.shape,Y_test.shape)

# 모델구성하기
model= Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(units=2,activation='relu'))
model.add(Dense(units=10,activation='softmax'))
model.summary()
# 모델 학습과정 설정
model.compile(loss='categorical_crossentropy',optimizer = 'sgd',metrics=['accuracy'])
# 모델 학습(콜백 적용)
from tensorflow.keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(patience=20) # val_accuracy가 연달아 2번이상 성능이 좋아지지않으면 stop(loss,accuracy,val_loss,val_accuracy)
hist = model.fit(X_train, Y_train , epochs=2000,batch_size=100,
                validation_data=(X_val,Y_val),
                callbacks=[earlyStopping])
